In [1]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, LineString, MultiLineString
from shapely.ops import unary_union
import numpy as np
from shapely.wkt import loads

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])

<ipython-input-3-d782d924483b>:1: DtypeWarning: Columns (94,98,101,115) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])


In [4]:
def parse_point(point_str):
    try:
        #文字列になっているから必要な数値だけ取得（POINT Z）を削除
        point_str = point_str[9:-2]

        #空白で分けx, yに代入
        x_str, y_str = point_str.strip().split()
        #ポイントに変換
        return Point(float(x_str), float(y_str))

        #エラーの場合は欠損地を入力
    except ValueError:
        return np.nan

#各行に対して関数を用いるときはapplyを用いる。Seriesだとエラーが起こることが多い
gdf_hole['geometry_x'] = gdf_hole['geometry_x'].apply(parse_point)

In [5]:
#dataframeをgeodataframeに変換
gdf_hole = GeoDataFrame(gdf_hole, geometry='geometry_x')

In [6]:
#空間結合した後にgeomeryを加えられるように連番を追加
number = [i for i in range(len(gdf_hole))]
gdf_hole['number'] = number

In [7]:
gdf_width = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/道路/2021/RdCL/14.csv') #このデータは道路中心線

<ipython-input-7-00602a3abf5e>:1: DtypeWarning: Columns (31,32,33,34,41,45,46,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf_width = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/道路/2021/RdCL/14.csv') #このデータは道路中心線


In [8]:
#WKTをデータフレーム化
gdf_width['geometry'] = loads(gdf_width['WKT'])#shapely.wktのloadsで文字列をラインデータに変換

In [9]:
#dataframeからgeodataframeに変換
gdf_width = GeoDataFrame(gdf_width, geometry='geometry')

In [10]:
if gdf_hole.crs is None:
    # 座標系をEPSG:6677に設定
    gdf_hole.set_crs(epsg=6677, inplace=True)

In [13]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6677  # 変換後の座標系を指定

gdf_width.crs = f'epsg:{src_proj}'
gdf_width = gdf_width.to_crs(epsg=dst_proj)

In [14]:
#gdf_widthを高速道路のみに絞る
gdf_width_highway = gdf_width[gdf_width['rdCtg'] == '高速自動車国道等']

# #gdf_osyu_width_highwayのindexを並び替える
gdf_width_highway.reset_index(drop=True, inplace=True)

In [18]:
#高速道路のデータフレームの頭にhighwayをつける

#すべてのカラムを取得
highway_columns = gdf_width_highway.columns.to_list()

#ループで各カラムを取得
for i in highway_columns[:56]:
  new_column = 'highway_' + i
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)

<ipython-input-18-81fd8ffd6c6e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-18-81fd8ffd6c6e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-18-81fd8ffd6c6e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new

In [19]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_hole, gdf_width_highway, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_width_highway[['highway_rID', 'geometry']], on='highway_rID')
# #geometryの名前を変更
result.rename(columns={'geometry':'nearest_highway'}, inplace=True)
#メモリの解放
del gdf_width

In [21]:
#dictance_highwayカラムを追加
result['distance_highway'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_highway'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_highway'][50*(i+1):]))

        result['distance_highway'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-21-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-21-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-21-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [25]:
#重複を削除
result = result[~result.number.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

#numberはいらないのでdrop
result.drop(columns=['index', 'geometry_x', 'index_road', 'number'], axis=1, inplace=True)

<ipython-input-25-a1d512f6b130>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(columns=['index', 'geometry_x', 'index_road', 'number'], axis=1, inplace=True)


In [29]:
result_columns = result.columns.to_list()
result[result_columns].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/横浜市　最近傍高速道路/横浜市_高速道路.csv', index=False)